# 2. Modeling

### 2.0 Load Train, Val, Test (Numpy Array)

- Gray Scale : Data_gray 폴더 안에 .npy 파일이 있으면 된다.
- RGB Scale : Data 폴더 안에 .npy 파일이 있으면 된다.

In [5]:
data_dir = 'Data_class_integrate_4'

In [6]:
!ls {data_dir}

X_test.npy  X_train.npy  X_val.npy  y_test.npy	y_train.npy  y_val.npy


#### Load Data

In [7]:
import os
import numpy as np

if os.path.exists(data_dir):
    X_train = np.load(data_dir + '/X_train.npy')
    y_train = np.load(data_dir + '/y_train.npy')
    X_val = np.load(data_dir + '/X_val.npy')
    y_val = np.load(data_dir + '/y_val.npy')
    X_test = np.load(data_dir + '/X_test.npy')
    y_test = np.load(data_dir + '/y_test.npy')

    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)
    print(X_test.shape, y_test.shape)

(300, 64, 128, 128, 3) (300, 4)
(100, 64, 128, 128, 3) (100, 4)
(100, 64, 128, 128, 3) (100, 4)


### 정규화

In [8]:
X_train = X_train.astype(float) / 255
X_val = X_val.astype(float) / 255
X_test = X_test.astype(float) / 255

### 2.1 MobileNet + GRU

In [59]:
MODELNAME='rgb_basic_mobilenet_gru'

CLASSES = 4
SIZE = (128, 128)
CHANNELS = 3 # RGB scale
NBFRAME = 64 
BS = 16

SPLIT_RATIO = (.2, .1)

In [60]:
epochs = 10
batch_size = 8

In [61]:
SIZE + (CHANNELS,)

(128, 128, 3)

### 2.1.1 Model Define

In [65]:
from keras.applications.mobilenet import MobileNet
from keras import Sequential
from keras.layers import TimeDistributed, GRU, Dense, Dropout, GlobalAveragePooling2D

def build_mobilenet(shape=SIZE + (CHANNELS,)):
    model = MobileNet(include_top=False,
                      input_shape=shape,
                      weights='imagenet')

    # Keep 9 layers to train
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
    
    output = GlobalAveragePooling2D()

    return Sequential([model, output])

In [68]:
def action_model(shape=(NBFRAME,) + SIZE + (CHANNELS,), nbout=CLASSES):
    # Create our convnet with (128, 128, 1) input shape
    mobilenet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = Sequential()

    # add the convnet with (64, 128, 128, 3) shape
    model.add(TimeDistributed(mobilenet, input_shape=shape))
    
    # here, you can also µuse GRU or LSTM
    model.add(GRU(64))
    
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))

    return model

In [69]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (64, 128, 128, 3)

model = action_model(INSHAPE, CLASSES)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 64, 1024)          3228864   
_________________________________________________________________
gru (GRU)                    (None, 64)                209280    
_________________________________________________________________
dense (Dense)                (None, 1024)              66560     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

### 2.1.2 Model Compile

In [9]:
from keras import optimizers

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.Adam(lr=0.0001), # defalut : 0.001
              metrics = ['accuracy'])

### 2.1.3 Model Fit

In [ ]:
%%time

History = model.fit(X_train, y_train,
                   epochs = epochs,
                   batch_size = batch_size,
                   validation_data = (X_val, y_val))

Epoch 1/10
38/38 [==============================] - 651s 17s/step - loss: 1.4167 - accuracy: 0.2703 - val_loss: 1.3601 - val_accuracy: 0.3200
Epoch 2/10
38/38 [==============================] - 649s 17s/step - loss: 1.3396 - accuracy: 0.3785 - val_loss: 1.2766 - val_accuracy: 0.3600
Epoch 3/10
38/38 [==============================] - 1048s 28s/step - loss: 3.8474 - accuracy: 0.4231 - val_loss: 1.3876 - val_accuracy: 0.3500
Epoch 4/10
38/38 [==============================] - 951s 25s/step - loss: 1.3303 - accuracy: 0.3067 - val_loss: 1.3037 - val_accuracy: 0.3800
Epoch 5/10
38/38 [==============================] - 1098s 29s/step - loss: 1.2828 - accuracy: 0.4540 - val_loss: 1.2759 - val_accuracy: 0.4400
Epoch 6/10
12/38 [========>.....................] - ETA: 11:37 - loss: 1.2661 - accuracy: 0.4663

### 2.1.4 Fitted Model Visualization

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(History.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, History.history['loss'])
plt.plot(epochs, History.history['val_loss'])

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(History.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, History.history['accuracy'])
plt.plot(epochs, History.history['val_accuracy'])

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.grid()
plt.show()

### 2.1.5 Model Evaluate

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test,
                                batch_size = batch_size)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))

### 2.1.6 Model Save

In [ ]:
save_dir = 'Models'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

final_model_name = '/' + scale + '_' + model_name + '.h5'

model.save(save_dir + final_model_name)

### 2.1.7 Saved Model Test

In [ ]:
from keras.models import load_model

lm = load_model(save_dir + final_model_name)

loss, accuracy = lm.evaluate(X_test, y_test,
                                batch_size = batch_size)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))